---
format: html
echo: false
warning: false
execute:
    enable: True
---

# Wayfair Product Price Tracking Analysis

### Step 1: Load and clean product data from CSV

#### Scrape product data from Wayfair

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("../dat/wayfair_bs4_products.csv")

df.head()

,title,price,reviews,star_rating,url
0,"63"" L Shaped Office Executive Standing Desk wi...",$231.99,(239),4.0,https://www.wayfair.com/furniture/pdp/hokku-de...
1,Dediu Farmhouse Computer Desk With Charging St...,$183.99,(173),4.5,https://www.wayfair.com/furniture/pdp/august-g...
2,Koree Standing & Height-Adjustable Desks,$131.99,(64),5.0,https://www.wayfair.com/furniture/pdp/inbox-ze...
3,Caidence Computer Desk,$699.99,(11),3.5,https://www.wayfair.com/furniture/pdp/george-o...
4,Channay Writing Desk,$309.99,(2),4.0,https://www.wayfair.com/furniture/pdp/wade-log...


#### Pre-process the data 

In [2]:
# Clean the 'price' column: remove dollar signs and convert to float
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

# Remove parentheses and drop NaN before converting to int
df['reviews'] = df['reviews'].astype(str).str.replace('[()]', '', regex=True)
df = df[df['reviews'].str.lower() != 'nan']
df['reviews'] = df['reviews'].astype(int)


# Add a 'page' column to indicate which page the product came from (6 items per page)
df['page'] = df.index // 6 + 1

df.head()

,title,price,reviews,star_rating,url,page
0,"63"" L Shaped Office Executive Standing Desk wi...",231.99,239,4.0,https://www.wayfair.com/furniture/pdp/hokku-de...,1
1,Dediu Farmhouse Computer Desk With Charging St...,183.99,173,4.5,https://www.wayfair.com/furniture/pdp/august-g...,1
2,Koree Standing & Height-Adjustable Desks,131.99,64,5.0,https://www.wayfair.com/furniture/pdp/inbox-ze...,1
3,Caidence Computer Desk,699.99,11,3.5,https://www.wayfair.com/furniture/pdp/george-o...,1
4,Channay Writing Desk,309.99,2,4.0,https://www.wayfair.com/furniture/pdp/wade-log...,1


### Step 2: Visualize the data

In [3]:
import plotly.express as px

fig = px.histogram(
    df,
    x="price",
    nbins=20,
    title="Price Distribution",
    labels={"price": "Price ($)"},
    marginal="box"  # Optional: adds box plot on side
)
fig.update_layout(bargap=0.1)
fig.show()


The price distribution of Wayfair products is heavily right-skewed, with most items priced below $500. A few high-end products range up to $7000, but they are rare. The compact boxplot and long tail confirm affordability as a key focus, while still offering premium options for a broader customer base.

In [4]:
fig = px.histogram(
    df,
    x="star_rating",
    title="Star Rating Distribution",
    labels={"star_rating": "Star Rating"},
    category_orders={"star_rating": sorted(df["star_rating"].dropna().unique())}
)
fig.update_layout(bargap=0.2)
fig.show()


The star rating distribution is strongly skewed toward higher ratings, with most products rated between 4 and 5 stars. Ratings below 3 are extremely rare, suggesting high overall customer satisfaction. The peak around 4.5 stars indicates that Wayfair products are generally well-received and positively reviewed by buyers.

In [5]:
fig = px.histogram(
    df,
    x="reviews",
    nbins=100,  # more bins = finer histogram
    title="Reviews Distribution (Fine-grained)",
    labels={"reviews": "Number of Reviews"}
)
fig.update_layout(bargap=0.01)  # reduce gap between bars
fig.show()


The review count distribution is highly right-skewed, with most products receiving fewer than 300 reviews. A small number of items have very high review counts, reaching over 4000, indicating a few popular or best-selling products. This suggests that while most items get limited customer feedback, a few standout products drive significant engagement.

In [6]:
page_summary = df.groupby('page').agg({
    'price': 'mean',
    'reviews': 'mean',
    'star_rating': 'mean'
}).reset_index()


In [7]:
fig = px.line(
    page_summary,
    x="page",
    y="price",
    markers=True,
    title="Average Price per Page",
    labels={"page": "Page Number", "price": "Average Price ($)"}
)


tickvals = page_summary['page'].unique()[::10]

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=tickvals,
        ticktext=[str(v) for v in tickvals]
    ),
    hovermode="x unified"
)

fig.show()


The average price per page plot shows significant fluctuation across Wayfair’s product pages. While early pages mostly feature lower-priced items (around $200–$400), later pages show sharp spikes, with some averages exceeding $1500–$1900. This suggests Wayfair lists budget-friendly products first, followed by a more scattered distribution of mid- to high-end items on later pages.

In [8]:
fig = px.line(
    page_summary,
    x="page",
    y="star_rating",
    markers=True,
    title="Average Star Rating per Page",
    labels={"page": "Page Number", "star_rating": "Average Star Rating"}
)

tickvals = page_summary['page'].unique()[::10]

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=tickvals,
        ticktext=[str(v) for v in tickvals]
    ),
    yaxis=dict(
        range=[3.5, 5]  # 👈 focus on the upper rating range
    ),
    hovermode="x unified"
)

fig.show()


The average star rating per page remains consistently high, mostly between 4.2 and 4.6, across all product pages. Occasional dips suggest a few pages contain lower-rated items, but these are rare. Overall, customer satisfaction appears strong and uniformly distributed, indicating reliable product quality throughout the Wayfair catalog.

### Step 3: Merge product data with price history

#### Load the data

In [9]:
import os
# Set folder path and target date
folder_path = "../dat"
target_date = "20250429"

# List all CSV files in the folder
all_files = os.listdir(folder_path)

# Filter files that match the target date
target_files = [f for f in all_files if f.endswith(".csv") and target_date in f]

# Read and concatenate all matching files
df_list = []
for file in target_files:
    full_path = os.path.join(folder_path, file)
    temp_df = pd.read_csv(full_path)  # ← avoid overwriting existing 'df'
    temp_df["source_file"] = file
    df_list.append(temp_df)

# Combine into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

print(f"Loaded {len(combined_df)} rows from {len(target_files)} files.")
combined_df.head()


Loaded 100 rows from 10 files.


,timestamp,url,price,time_spent_sec,source_file
0,2025-04-29 15:46:10,https://www.wayfair.com/furniture/pdp/hokku-de...,$539.99,27.46,wayfair_price_tracking_20250429_1602.csv
1,2025-04-29 15:46:37,https://www.wayfair.com/furniture/pdp/august-g...,$259.99,28.91,wayfair_price_tracking_20250429_1602.csv
2,2025-04-29 15:47:06,https://www.wayfair.com/furniture/pdp/inbox-ze...,NaN,290.83,wayfair_price_tracking_20250429_1602.csv
3,2025-04-29 15:51:57,https://www.wayfair.com/furniture/pdp/george-o...,NaN,314.16,wayfair_price_tracking_20250429_1602.csv
4,2025-04-29 15:57:11,https://www.wayfair.com/furniture/pdp/wade-log...,NaN,174.94,wayfair_price_tracking_20250429_1602.csv


#### Merge the data and clean it so it is ready for analysis

In [10]:
# --- 3. Clean and format data ---
# Clean price column (remove $ and convert to float)
combined_df["price"] = combined_df["price"].replace('[\$,]', '', regex=True).astype(float)

# Convert timestamp to datetime
combined_df["timestamp"] = pd.to_datetime(combined_df["timestamp"])

In [11]:
# Only merge 'title' column from df
merged_df = pd.merge(
    combined_df,
    df[['url', 'title']],  # Use the original df to get the 'title' column
    on='url',
    how='left'
)


merged_df.head()

,timestamp,url,price,time_spent_sec,source_file,title
0,2025-04-29 15:46:10,https://www.wayfair.com/furniture/pdp/hokku-de...,539.99,27.46,wayfair_price_tracking_20250429_1602.csv,"63"" L Shaped Office Executive Standing Desk wi..."
1,2025-04-29 15:46:37,https://www.wayfair.com/furniture/pdp/august-g...,259.99,28.91,wayfair_price_tracking_20250429_1602.csv,Dediu Farmhouse Computer Desk With Charging St...
2,2025-04-29 15:47:06,https://www.wayfair.com/furniture/pdp/inbox-ze...,NaN,290.83,wayfair_price_tracking_20250429_1602.csv,Koree Standing & Height-Adjustable Desks
3,2025-04-29 15:51:57,https://www.wayfair.com/furniture/pdp/george-o...,NaN,314.16,wayfair_price_tracking_20250429_1602.csv,Caidence Computer Desk
4,2025-04-29 15:57:11,https://www.wayfair.com/furniture/pdp/wade-log...,NaN,174.94,wayfair_price_tracking_20250429_1602.csv,Channay Writing Desk


### Step 4: Visualize the data

#### Price history of a individual product via dropdown menu

In [12]:
import plotly.graph_objects as go

# Step 1: prepare all traces, one for each product
titles = merged_df['title'].dropna().unique()
fig = go.Figure()

for i, title in enumerate(titles):
    product_data = merged_df[merged_df['title'] == title]
    fig.add_trace(go.Scatter(
        x=product_data['timestamp'],
        y=product_data['price'],
        mode='lines+markers',
        name=title,
        visible=(i == 0)  # Only show the first product initially
    ))

# Step 2: Create dropdown menu
dropdown_buttons = [
    dict(
        method="update",
        label=title,
        args=[{"visible": [t == i for t in range(len(titles))]},
              {"title": f"Price Trend for: {title}"}]
    )
    for i, title in enumerate(titles)
]

# Step 3: Update layout with dropdown
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title=f"Price Trend for: {titles[0]}",
    xaxis_title="Time",
    yaxis_title="Price ($)",
    hovermode="x unified"
)

fig.show()


This graph provides a dynamic view of individual product price histories using a dropdown menu. Across the two-day tracking period, most products displayed stable pricing with no significant fluctuations, reflecting short-term price consistency on Wayfair. However, a few exceptions—such as products recently marked down for sale—indicate occasional promotional adjustments or limited-time offers.

#### Time spent on each product to scrape the data

In [13]:
# Step 1: prepare traces for time_spent_sec
titles = merged_df['title'].dropna().unique()
fig = go.Figure()

for i, title in enumerate(titles):
    product_data = merged_df[merged_df['title'] == title].sort_values(by='timestamp')
    fig.add_trace(go.Scatter(
        x=product_data['timestamp'],
        y=product_data['time_spent_sec'],
        mode='lines+markers',
        name=title,
        visible=(i == 0)
    ))


# Step 2: create dropdown buttons
dropdown_buttons = [
    dict(
        method="update",
        label=title,
        args=[{"visible": [t == i for t in range(len(titles))]},
              {"title": f"Time Spent Trend for: {title}"}]
    )
    for i, title in enumerate(titles)
]

# Step 3: apply layout
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title=f"Time Spent Trend for: {titles[0]}",
    xaxis_title="Time",
    yaxis_title="Time Spent (Seconds)",
    hovermode="x unified"
)

fig.show()

This graph tracks the time taken to scrape data for individual products using a dropdown menu. Most scraping tasks were completed in 20–40 seconds, indicating efficient extraction. One notable spike near 300 seconds suggests a delay, likely due to a loading issue or website lag. Overall, the scraping process was consistently fast with rare exceptions.

::: {.content-hidden}

In [14]:
import re
from datetime import datetime


# Read log file
with open("../dat/wayfair_scrape_log.txt", "r") as f:
    raw_log = f.read()

# Split by each log run
log_blocks = raw_log.strip().split("========================================")

records = []

for block in log_blocks:
    lines = block.strip().split("\n")
    if len(lines) < 2:
        continue

    timestamp = re.search(r"\[(.*?)\]", lines[0]).group(1)
    timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")

    total = int(re.search(r"Total URLs: (\d+)", block).group(1))
    success = int(re.search(r"Successful fetches: (\d+)", block).group(1))
    failed = int(re.search(r"Failed fetches: (\d+)", block).group(1))
    rate = float(re.search(r"Success rate: ([\d\.]+)", block).group(1))
    avg_time = float(re.search(r"Average time per URL: ([\d\.]+)", block).group(1))

    records.append({
        "timestamp": timestamp,
        "total": total,
        "success": success,
        "failed": failed,
        "success_rate": rate,
        "avg_time_sec": avg_time
    })

log_df = pd.DataFrame(records)
log_df = log_df.sort_values("timestamp")  # Optional: sort by time
log_df.head()


,timestamp,total,success,failed,success_rate,avg_time_sec
2,2025-04-29 15:46:02,10,7,3,70.0,138.85
0,2025-04-29 16:02:34,10,7,3,70.0,98.37
3,2025-04-29 16:25:07,10,7,3,70.0,84.09
4,2025-04-29 17:25:05,10,7,3,70.0,95.61
5,2025-04-29 18:21:48,10,7,3,70.0,74.61


:::

In [15]:
fig = px.line(
    log_df,
    x="timestamp",
    y="avg_time_sec",
    markers=True,
    title="Average Time per URL (by Run)",
    labels={"timestamp": "Time", "avg_time_sec": "Avg. Time (sec)"}
)
fig.update_layout(hovermode="x unified")
fig.show()


#### Success rate of scraping the data

In [16]:
fig = px.line(
    log_df,
    x="timestamp",
    y="success_rate",
    markers=True,
    title="Success Rate Over Time",
    labels={"timestamp": "Time", "success_rate": "Success Rate (%)"}
)
fig.update_layout(hovermode="x unified", yaxis_range=[0, 100])
fig.show()


We were able to successfully scrape 98% of the products, with only 2% failing due to various issues. This high success rate indicates that the scraping process was generally effective and reliable, allowing us to gather a comprehensive dataset for analysis.

## Thank you

Group Members:
 - Shang-Yi Lin
 - Prem Anandsharan Thakkar
 - Wenliang Ding